In [ ]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt

import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
data = yf.download(ticker, period="1y", interval="1h")


In [ ]:
data

In [ ]:
# Importing through a CSV that has more data which was downloaded externally.
df = pd.read_csv('../Data/AUDCAD_H1(22007-09-26 - 2024-02-02).csv', delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0)
df.index = pd.to_datetime(df.index)


In [ ]:
# csv slice by date
start_date = '2010-01-01'
end_date = '2024-01-01'
df = df.loc[start_date:end_date]

# Write out strategy here to figure out the logic

In [ ]:
class Strat(Strategy):
    def init(self):
        pass

    def next(self):
        pass

bt = Backtest(df_slice, Strat, cash=10_000)
bt.run()
bt.plot()

In [ ]:
# help(ta.cdl_pattern(name='engulfing'))
help(ta.cdl_engulfing)

In [20]:
class Strat(Strategy):
    def init(self):
        # Calculate the 50-period EMA and 14-period ATR and check engulfing        
        self.ema = self.I(ta.ema, pd.Series(self.data.Close), length=50)
        self.atr = self.I(ta.atr, 
                          pd.Series(self.data.High), 
                          pd.Series(self.data.Low), 
                          pd.Series(self.data.Close), 
                          length=14, plot=False)
               
        self.pullback = False
        self.pullback_count = 0
        self.consolidation_high = 0
        
    def next(self):
        current_ema = self.ema[-1]
        
        current_close = self.data.Close[-1]
        current_open = self.data.Open[-1]
        current_high = self.data.High[-1]

        bull_body = self.data.Close[-1] > self.data.Open[-1]
        bear_body = self.data.Close[-1] < self.data.Open[-1]

        
        # Reset after a fresh crossover:
        if crossover(current_close, current_ema) or (current_close < current_ema):
            self.pullback = False
            self.pullback_count = 0
            self.consolidation_high = 0

        if current_high > self.consolidation_high:
            self.consolidation_high = current_high
        
        if current_close < self.consolidation_high and current_close < current_open:
            self.pullback_count += 1
        if self.pullback_count >= 2:
            self.pullback = True
        if current_close < self.consolidation_high and self.pullback:
            if bull_body and current_close > self.data.High[-2]:
                self.buy(sl=(self.data.Low[-1] - self.atr[-1]), tp=(self.data.High + (self.atr[-1] * 1.2)))
                self.pullback = False
                self.pullback_count = 0
                self.consolidation_high = 0

In [21]:
bt = Backtest(data, Strat, cash=100000)
stats = bt.run()
print(stats)
bt.plot()

Start                     2023-02-07 21:00...
End                       2024-02-07 21:00...
Duration                    365 days 00:00:00
Exposure Time [%]                   34.380596
Equity Final [$]                 98215.342698
Equity Peak [$]                 102145.103897
Return [%]                          -1.784657
Buy & Hold Return [%]               13.057013
Return (Ann.) [%]                   -1.717129
Volatility (Ann.) [%]                4.297429
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                   -5.347798
Avg. Drawdown [%]                   -0.718019
Max. Drawdown Duration      351 days 00:00:00
Avg. Drawdown Duration       35 days 17:00:00
# Trades                                  230
Win Rate [%]                        56.086957
Best Trade [%]                       0.569179
Worst Trade [%]                     -1.066632
Avg. Trade [%]                    

d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
d:\Apps\Backtesting\sandbox\venv\Lib\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p3421', ...)

In [ ]:
stats._trades

In [ ]:

# plot own equity curve based on backtesting data for data more than 10K
equity_curve = stats._equity_curve['Equity']

plt.figure(figsize=(10, 6))
plt.plot(equity_curve, label='Equity Curve', lw=1)  # lw is line width
plt.title('Equity Curve')
plt.xlabel('Time')
plt.ylabel('Equity')
plt.legend()
plt.grid(False)
plt.show()


In [ ]:

equity_curve = stats._equity_curve['Equity'].reset_index()

fig = px.line(equity_curve, x='index', y='Equity', labels={'index': 'Time'}, title='Strategy Performance')
fig.update_layout(height=600, xaxis_title='Time', yaxis_title='Equity', legend_title='Legend')
fig.show()
